# Getting Started with PrimeHub Python SDK
PrimeHub Python SDK makes you automation with PrimeHub Platform.

In order to make the SDK working, you have to

- install the library with pip
- create a config file in the ~/.primehub/config.json

# Part 1: prerequisite: Configure the environment.

## 1. Install with pip
Let's install PrimeHub Python SDK with pip.

In [1]:
!pip install primehub-python-sdk

## 2. Request the API Token
In order to get the token, you have to have an account in the PrimeHub cluster, the following process will ask you loing with your account.



In [2]:
# PLEASE UPDATE PRIMEHUB_CLUSTER to your cluster
PRIMEHUB_CLUSTER = 'http://primehub-python-sdk.primehub.io'

In [3]:
import os
from primehub import PrimeHub, PrimeHubConfig

ph = PrimeHub(PrimeHubConfig())
if not os.path.isfile(os.path.join(os.getenv("HOME"), ".primehub/config.json")):
    ph.config.generate(PRIMEHUB_CLUSTER)

In [4]:
ph = PrimeHub(PrimeHubConfig())
if ph.is_ready():
    print("PrimeHub Python SDK setup successfully")
    print("Current Group:", ph.primehub_config.current_group)
else:
    print("PrimeHub Python SDK couldn't get the group information, please check the configuration.")

PrimeHub Python SDK setup successfully
Current Group: {'id': 'ccdd52b8-db16-46ce-8a06-70343c79e82a', 'name': 'phusers', 'displayName': 'primehub users'}


## 3. Check the account is Admin account

Use `ph.me.me` to know that the account is admin account.

In [5]:
account_information = ph.me.me()
account_information['isAdmin']

True

## 4. Clean up the previous testing data.

We need to clean up the previous data to prevent the notebook error,

In [6]:
import pandas as pd

In [7]:
# Clean up "test_group_from_jupyter" group data.

df_list = pd.DataFrame.from_records(list(ph.admin.groups.list()))
if len(df_list) > 0:
    target_id = list(df_list.loc[df_list['name'] == "test_group_from_jupyter"]["id"])

    if len(target_id) == 1:
        print("clean the group data: {}".format(target_id))
        ph.admin.groups.delete(list(target_id)[0])

clean the group data: ['df7fdfe4-5e12-4f1c-aeac-8004160c8e86']


In [8]:
# Clean up "user-admin-from-jupyter" user data.

df_list = pd.DataFrame.from_records(list(ph.admin.users.list()))
if len(df_list) > 0:
    target_id = list(df_list.loc[df_list['username'] == "user-admin-from-jupyter"]["id"])

    if len(target_id) == 1:
        print("clean the user data: {}".format(target_id))
        ph.admin.users.delete(list(target_id)[0])

# Part 2: User-wise operation
We will test:
    
- create a user
- Enabling Personal Volume Capacity with specified xxG
- Assign the user multiple groups
- Get a group list of the user

## 1. Create a group with admin role

Use `ph.admin.groups.create` to create the group.

In [9]:
# Create a group with admin role
config = {
  "name": "test_group_from_jupyter",
  "displayName": "test_group",
  "enabledDeployment": False,
  "enabledSharedVolume": False,
  "quotaCpu": 0.5,
  "quotaGpu": 0,
  "admins": "",
  "users": {
      "connect": []
  }
}
data = ph.admin.groups.create(config)

In [10]:
data['id']

'd30688b3-fdf3-4b28-84ac-a1ef29c01033'

## 2. Create a user with admin role

Use `ph.admin.users.create` to :
- create the user
- enable PV capacity with specified 2Gb
- Assign the user multiple groups

In [11]:
# Create a user with admin role
config = {
  "username": "user-admin-from-jupyter",
  "groups": {
    "connect": [
      {
        "id": ph.group_id
      },
      {
        "id": data['id']
      }
    ]
  },
  "isAdmin": True,
  "volumeCapacity": 2
}
data = ph.admin.users.create(config)

## 3. Get a group list of the user

Use `ph.admin.groups.list` and `ph.admin.users.list()` to get the group and user list.

In [12]:
import pandas as pd

In [13]:
pd.DataFrame.from_records(list(ph.admin.groups.list()))

,id,displayName,name,admins,quotaCpu,quotaGpu,quotaMemory,projectQuotaCpu,projectQuotaGpu,projectQuotaMemory,sharedVolumeCapacity
0,ccdd52b8-db16-46ce-8a06-70343c79e82a,primehub users,phusers,phadmin,NaN,NaN,None,None,None,None,1.0
1,d30688b3-fdf3-4b28-84ac-a1ef29c01033,test_group,test_group_from_jupyter,,0.5,0.0,None,None,None,None,NaN


In [14]:
ph.admin.groups.get(ph.group_id)['users']

[{'id': 'bf989d98-cafc-47a6-832b-2d9843761cf7', 'username': 'phadmin'},
 {'id': '8a15793c-5845-44b1-9efa-0f1ca858fc1e',
  'username': 'user-admin-from-jupyter'}]